# Asesõnade leidmine lausetest

Kood, mis vajab sisendiks faili '100000_random_sentences.csv' ja annab väljundiks faili 'asesonad_lausetes.xlsx'. \
\
Sisendfail '**100000_random_sentences.csv**', sisaldab 100 000 juhuslikku lauset eesti keele koondkorpusest. Failis olevad andmed on jaotatud tulpadesse 'sentence_id', 'document_id', 'sentence_start', 'sentence_end', 'subcorpus' ja 'text'. Antud töö jaoks huvitab meid ainult tulp 'text', mis sisaldab lauseid. \
\
Väljundfail '**asesonad_lausetes.xlsx**', sisaldab 10 000 suvalist sisendfailist saadud lauset, milles leidub asesõna/asesõnu.
Tulpade tähendused:
* **Index** - lause järjekorra index (al 1),
* **pronoun** - lauses leiduv asesõna,
* **pronoun lemma** - lauses leiduva asesõna lemma,
* **pronoun sentence** - lause, kus selles leiduv asesõna on ümbritsetud topelt tärnidega - **<asesõna>**.


In [1]:
# Vajalike teekide import
import estnltk
from estnltk import Text
import pandas as pd
import csv

In [2]:
# Funktsioon, mis võtab vastu Text-tüüpi objekti ja tagastab selles leiduvad asesõnad, nende lemmad ja 
# nende morfoloogilise analüüsi
def get_pronoun_info(row):
    pronoun_text = [word.text for word in row.words if 'P' in word.morph_analysis.partofspeech]
    pronoun_lemma = [word.morph_analysis.lemma for word in row.words if 'P' in word.morph_analysis.partofspeech]
    pronoun_morph_analysis = [word.morph_analysis for word in row.words if 'P' in word.morph_analysis.partofspeech]
    return pronoun_text, pronoun_lemma, pronoun_morph_analysis

In [3]:
# Funktsioon, mis võtab vastu Dataframe'i rea ja tagastab 'object' ja 'pronoun morph' tulbast kokku pandud lause, 
# kus asesõna on ära tähistatud formaadis -**<asesõna>**
# 'object' tulp sisaldab morfoloogilise analüüsiga lauseid
# 'pronoun morph' tulp sisaldab morfoloogilise analüüsiga lauses leiduvat asesõna
def mark_pronoun(row):
        annotated_text = "{} **{}** {}".format(
        row['object'].text[:row['pronoun morph'].start],
        row['pronoun morph'].text,
        row['object'].text[row['pronoun morph'].end:])
        return annotated_text

In [6]:
# Loeb lausete faili sisse
input=pd.read_csv("100000_random_sentences.csv")

In [8]:
# Teeb lausete tulbast koopia tulemuste tabeli jaoks
result = input[['text']].copy()

In [9]:
# Loob juurde morfoloogilise analüüsiga märgendatud lausete tulba
result['object'] = result['text'].apply(lambda text: Text(text).tag_layer('morph_analysis'))

In [10]:
# Loob juurde asesõnade teksti, lemma ja morfoloogilise analüüsi tulba kasutades funktsiooni 'get_pronoun_info(text)'
result[['pronoun','pronoun lemma', 'pronoun morph']] = result['object'].apply(lambda text: pd.Series(get_pronoun_info(text)))

In [11]:
# Eemaldame read lausetega, milles ei leidu asesõnu
result= result[result['pronoun'].map(lambda x: len(x)) > 0]

In [ ]:
# Võtab 10 000 suvalist kirjet
result = result.sample(n=10000, random_state=32)

In [ ]:
# Jagab iga asesõna (ja selle lemma ja morfoloogilise analüüsi) eraldi reale
result = result.explode(['pronoun', 'pronoun lemma', 'pronoun morph'])

In [ ]:
# Loob juurde tulba, kus asesõna on lauses tähistatud
result['pronoun sentence'] = result.apply(mark_pronoun, axis=1)

In [ ]:
# Lisab tulemuste tabelile indexi tulba
result['Index'] = range(1, len(result) + 1)
result.set_index('Index', inplace=True)

In [ ]:
# Eemaldab ebavajalikud tulbad
result = result.drop(['text', 'object', 'pronoun morph'], axis=1) 

In [ ]:
# Salvestab saadud tulemuste tabeli faili 'asesonad_lausetes.xlsx'
writer = pd.ExcelWriter("asesonad_lausetes.xlsx", engine="xlsxwriter", engine_kwargs={"options": {"strings_to_formulas": False}},
)

with writer as writer:
    result.to_excel(writer)